# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [3]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/Mragank/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [21]:
from nltk.corpus import brown
brown.words()


[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Mragank/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

processed_corpus=[]
for words in brown.sents():
    sentence=' '.join(words)
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    processed_corpus.append(filtered_words)
# print processed_corpus[0]
# print len(processed_corpus)



In [40]:
import operator

word_count={}
for sent in processed_corpus:
    for word in sent:
        word_count[word]=word_count.get(word,0)+1
sorted_words = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)
vocabulary=[]
context=[]
topTwenty=[]
for i in range(5000):
    vocabulary.append(sorted_words[i][0])
    if i <1000:
        context.append(sorted_words[i][0])
    if i <20:
        topTwenty.append(sorted_words[i][0])
print "Most Frequent 20 words in Corpus"
for i in topTwenty:       
    print i

Most Frequent 20 words in Corpus
one
would
said
time
new
could
two
may
first
man
like
even
made
also
many
must
well
af
back
years


## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [41]:
#building complete context for all vocabulary words
word_to_context={}
for sentence in brown.sents():
    for idx in range(len(sentence)):
        if sentence[idx].lower() in vocabulary:
            surroundings=word_to_context.get(sentence[idx].lower(),{})
            for window in range(max(idx-2,0),min(idx+3,len(sentence))):
                surroundings[sentence[window].lower()]=surroundings.get(sentence[window].lower(),0)+1
            word_to_context[sentence[idx].lower()]=surroundings

    

In [69]:
#building co-ocurrence matrix from the complete context dictionary created above
co_occurence_matrix=[[0 for i in range(len(context))]for j in range(len(vocabulary))]
nf_count=0
nf_d={}
for v in range(len(vocabulary)):
    for c in range(len(context)):
        raw_context=word_to_context.get(vocabulary[v],None)
        if raw_context:
            co_occurence_matrix[v][c]=raw_context.get(context[c],0)
        else:
            co_occurence_matrix[v][c]=0
# print nf_count  
# for i in range(10):
#     print co_occurence_matrix[i]
print "Done"

Done


## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [70]:
from __future__ import division
# creating both Pr(c|w) and Pr(c) matrix and array
pr_cw=[[0 for i in range(len(context))]for j in range(len(vocabulary))]
pr_c=[0 for i in range(len(context))]
total_context_frequency=0

#Added Laplace smoothing by 1 to avoid division by 0 issues
for v in range(len(vocabulary)):
    total_context_frequency+=sum(co_occurence_matrix[v])+len(context)
    for c in range(len(context)):
        pr_cw[v][c]=co_occurence_matrix[v][c]+1/(sum(co_occurence_matrix[v])+len(context))
for c in range(len(context)):
    for v in range(len(vocabulary)):
        pr_c[c]+= co_occurence_matrix[v][c]+1
    pr_c[c]/=total_context_frequency
    
# print pr_c


## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [71]:
import math
word_vectors=[[0 for i in range(len(context))]for j in range(len(vocabulary))]
for v in range(len(vocabulary)):
    for c in range(len(context)):
        word_vectors[v][c]=max(0,math.log10(pr_cw[v][c]/pr_c[c]))
# print word_vectors[1]
print "Done"

Done


## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [72]:
#clustering into 100 clusters
import numpy as np  
from sklearn.cluster import KMeans  
word_vectors = np.asarray(word_vectors)
kmeans = KMeans(n_clusters=100)  
kmeans.fit(word_vectors)
word_clusters={}
for label in range(len(kmeans.labels_)):
    cluster= word_clusters.get(kmeans.labels_[label],[])
    cluster.append(vocabulary[label])
    word_clusters[kmeans.labels_[label]]=cluster
# Uncomment this to print all clusters    
# for k,v in word_clusters.iteritems():
#     print word_clusters[k]

In [75]:
#Nearest Neighbours for top 20 most frequest words
from sklearn.neighbors import NearestNeighbors
neighbours= NearestNeighbors(n_neighbors=4, radius=1.0, 
    algorithm='auto', metric='cosine')
neighbours.fit(word_vectors)
top_twenty=[]
for i in range(20):
    top_twenty.append(word_vectors[i])
# print neighbours.kneighbors(top_twenty)
three_neighbours= neighbours.kneighbors(top_twenty, return_distance=False)
for i in range(20):
    print "Three nearest neighbours of \""+vocabulary[i]+"\":"
    print vocabulary[three_neighbours[i][1]],
    print vocabulary[three_neighbours[i][2]],
    print vocabulary[three_neighbours[i][3]]
    print
        


Three nearest neighbours of "one":
would new another

Three nearest neighbours of "would":
one could may

Three nearest neighbours of "said":
one would like

Three nearest neighbours of "time":
one would man

Three nearest neighbours of "new":
one would may

Three nearest neighbours of "could":
would one may

Three nearest neighbours of "two":
one many three

Three nearest neighbours of "may":
would one must

Three nearest neighbours of "first":
one would new

Three nearest neighbours of "man":
one time would

Three nearest neighbours of "like":
would one could

Three nearest neighbours of "even":
one would may

Three nearest neighbours of "made":
one would make

Three nearest neighbours of "also":
would one may

Three nearest neighbours of "many":
two one would

Three nearest neighbours of "must":
would may one

Three nearest neighbours of "well":
would one even

Three nearest neighbours of "af":
one may would

Three nearest neighbours of "back":
one would go

Three nearest neighbours

<b>Clustering Observation</b>

Note: The part of code mentioned with a comment can be uncommented to print all 100 clusters.
    
After clustering into 100 clusters and printing the words of all the 100 clusters, I was able to observe that
similar words were clustered together in one group.
We can see that with the help of following example clusters I found:

Example 1:
[u'black', u'red', u'brown', u'dark', u'kept'...]

All colors from the text were in same cluster

Example 2:
[u'1', u'2', u'3', u'per', u'4', u'5', u'10', u'figure', u'6', u'8', u'1960', u'7', u'30',
 u'cent', u'15', u'20', u'12', u'nearly', u'1961', u'50', u'25', u'100'..]:
 
 All numbers from the text were in the same cluster

Example 3:
[u'united', u'form', u'case', u'family', u'members'..]

All words related to group/collection of people were clustered in the same group.

There were also some words clutered together that were not very 
similar in meaning but I think the reason for that is pretty obvious. Words were clustered based on their cluster centers
that were derived from their vector representation. We only used 1000 context words and 5000 vocabulary to define 
every word as a vector. Not only is the size of the vector small in terms of dimension but also the vectors are sparse.
Also to determine the vector values, we used basic probability distribution which may not be the best measure to create
mutual information vectors.


<b>Nearest Neighbour Observation</b>

The results of 3 nearest neighbours of all the most frequent 20 words has been printed above. 
We observe that most of the neighbours are dominated by the top 100 or top 150 most frequent words in the corpus. I belive this may be because of the distance function that we have used to find the closest neighbours. We have cosine similarity which is highly biased towards the bigger vectors. Also the closest aren't totally irrelevant and some of them are proper but I believe that can be improved if I used a better distance function or used some normalisation technique to offset the effect of very frequent words.

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [1]:
import codecs
training_data=[]
testing_data=[]

#Reading Training Data
with codecs.open("/Users/mragank/Desktop/netflix-dataset/TrainingRatings.txt", encoding="utf-8") as f:
    try:
        for row in  f.readlines():
            training_data.append(row.split(','))
    finally:
        f.close()

#Reading Testing Data
with codecs.open("/Users/mragank/Desktop/netflix-dataset/TestingRatings.txt", encoding="utf-8") as f:
    try:
        for row in  f.readlines():
            testing_data.append(row.split(','))
    finally:
        f.close()

#Printing Training Data Details
tr_users={}
users_count=0
tr_movies={}
movies_count=0
tr_ratings={}
print "TRAINING SET"
print "Total Samples in Training Set="+str(len(training_data))
for sample in training_data:
    if tr_movies.get(sample[0],None)==None:
        tr_movies[sample[0]]=movies_count
        movies_count+=1
    if tr_users.get(sample[1],None)==None:
        tr_users[sample[1]]=users_count
        users_count+=1
    if tr_ratings.get(sample[2],None)==None:
        tr_ratings[sample[2]]=True
print "Total unique users in Training Set="+str(len(tr_users))
print "Total unique movies in Training Set="+str(len(tr_movies))
print "Total unique ratings in Training Set="+str(len(tr_ratings))
print

#Printing Testing Data Details
tst_users={}
users_count=0
tst_movies={}
movies_count=0
tst_ratings={}
print "TESTING SET"
print "Total Samples in Testing Set="+str(len(training_data))
for sample in testing_data:
    if tst_movies.get(sample[0],None)==None:
        tst_movies[sample[0]]=movies_count
        movies_count+=1
    if tst_users.get(sample[1],None)==None:
        tst_users[sample[1]]=users_count
        users_count+=1
    if tst_ratings.get(sample[2],None)==None:
        tst_ratings[sample[2]]=True
print "Total unique users in Testing Set="+str(len(tst_users))
print "Total unique movies in Testing Set="+str(len(tst_movies))
print "Total unique ratings in Testing Set="+str(len(tst_ratings))

TRAINING SET
Total Samples in Training Set=3255352
Total unique users in Training Set=28978
Total unique movies in Training Set=1821
Total unique ratings in Training Set=5

TESTING SET
Total Samples in Testing Set=3255352
Total unique users in Testing Set=27555
Total unique movies in Testing Set=1701
Total unique ratings in Testing Set=5


## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [2]:
#Creating User to Movies Rating Matrix from the input data
training_matrix=[[-1 for i in range(len(tr_movies))]for j in range(len(tr_users))]
testing_matrix=[[-1 for i in range(len(tst_movies))]for j in range(len(tst_users))]
for sample in training_data:
    training_matrix[tr_users[sample[1]]][tr_movies[sample[0]]]=float(sample[2])
    
for sample in testing_data:
    testing_matrix[tst_users[sample[1]]][tst_movies[sample[0]]]=float(sample[2])
print "Done"

Done


In [3]:
#Recommendation Algorithm
from __future__ import division
average_ratings=[]
for user in range(len(training_matrix)):
    va=0
    count=0
    for movie in range(len(training_matrix[0])):
        if training_matrix[user][movie]!=-1:
            va+=training_matrix[user][movie]
            count+=1
    average_ratings.append(va/count)
# print average_ratings
print "Done"

    

Done


In [5]:
real_rating=[]
predicted_rating=[]
for sample in range(5000):
    user= testing_data[sample][1]
    movie=testing_data[sample][0]
    real_rating.append(float(testing_data[sample][2]))
    
    if tr_users.get(user,None)!=None and tr_movies.get(movie,None)!=None:
        va = average_ratings[tr_users[user]]
        k=0.001
        temp=0
        for i in range(len(training_matrix)):
            wai=0
            vajss=0
            vijss=0
            for j in range(len(training_matrix[0])):
                if training_matrix[tr_users[user]][j]!=-1 and training_matrix[i][j]!=-1:
                    vaj=training_matrix[tr_users[user]][j]-average_ratings[tr_users[user]]
                    vij=training_matrix[i][j]-average_ratings[i]
                    vajss+= vaj**2
                    vijss+= vij**2
                    wai+=(vaj*vij)
            if vajss!=0 and vijss!=0:
                wai/= (vajss*vijss)**0.5
            curr_movie_rating=0
            if training_matrix[i][tr_movies[movie]]!=-1:
                curr_movie_rating=training_matrix[i][tr_movies[movie]]
                temp+=(wai*(curr_movie_rating-average_ratings[i]))
#         print wai,
#         print temp
        temp*=k
        pa=va+temp
        predicted_rating.append(pa)
                    
    else:
        predicted_rating.append(sum(average_ratings)/len(average_ratings))
    if sample%25==0:
        print ".",
# print predicted_rating
# print real_rating[:10]
print "Done"

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Done


## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [6]:
# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
print mean_absolute_error(real_rating, predicted_rating)

0.7248858378559189


In [7]:
# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
print mean_squared_error(real_rating, predicted_rating)

0.8540646831590417


## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

In [8]:
#K-Nearest neigbour Approach for Rating Prediction
from sklearn.neighbors import NearestNeighbors
neigh= NearestNeighbors(n_neighbors=100, radius=1.0, algorithm='auto', metric='cosine')
neigh.fit(training_matrix)

pr=[]
rr=[]
test_data=[]
movie_data=[]
for sample in range(5000):
    user= testing_data[sample][1]
    movie=testing_data[sample][0]
    rr.append(float(testing_data[sample][2]))
    test_data.append(training_matrix[tr_users[user]])
    movie_data.append(movie)
    
# print neighbours.kneighbors(top_twenty)
fifty_neighbours= neigh.kneighbors(test_data, return_distance=False)
# print fifty_neighbours
# print training_data[1]
for sample in range(len(fifty_neighbours)):
    rating=0
    count=0
    for neighbour in range(1,len(fifty_neighbours[sample])):
        if training_matrix[fifty_neighbours[sample][neighbour]][tr_movies[movie_data[sample]]]!=-1:
            rating+=training_matrix[fifty_neighbours[sample][neighbour]][tr_movies[movie_data[sample]]]
            count+=1
    if count!=0:
        pr.append(rating/count)
    else:
        pr.append(average_ratings[tr_users[user]])
print "Done"

Done


In [9]:
# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
print mean_absolute_error(rr, pr)

0.803930816596558


In [10]:
# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
print mean_squared_error(rr, pr)

1.027159146621389


To check for improvements on the pearson correlation approach implemented above, I tried two different approaches on the complete 5000 users data set. Here are the approaches and their results:

Approach 1: K-Means Clustering<br>
I formed 500 clusters based on user similarity. I used the user-item matrix to form user vectors and used those vectors to group similar users. Once the clusters were formed, I found the clusters of the user in question and checked how many users in the cluster have rated the movie in question. I calculated the average rating for that movie by similar users in the clusters and assigned that rating to the movie. Here are the MAE and RMSE values obtained from this approach:<br>
MAE: 0.846841002769792<br>
RMSE: 1.2108123167172067<br>

Approach 2: N-Nearest Neigbours<br>
Since the results by this approach weren't highly optimal, I tried the second approach of nearest neighbours. I again formed user vectors from user-item matrix and found the 100 nearest neighbours for every user being tested. Once I found the neighbours, I checked all the neighbours that have rated the movie in question and calculated the average movie rating for the givem movie. This approach significantly improved the MAE and RMSE values in comparison to the clustering techniques and the results are as follows:<br>
MAE: 0.803930816596558<br>
RMSE: 1.027159146621389<br>